##problem
1. as a doctor I need to be able easily see where my team is on a weekly basis 
2. as a doctor with responsibility for the weekly rota I need a way of generating a weekly view from the the master rota that is robust and time efficient
n.b the current approach of manually creating a weekly view of the master rota is prone to transcription error and a waste of clinical time

##approach
1. automate the task of creating a weekly view of the master rota with python
2. longer term, maybe a giant excel spreadsheet isn't the optimal master rota solution?

In [1]:
import pandas as pd
import datetime
import collections

In [2]:
def next_weekday(d, weekday):
    days_ahead = weekday - d.weekday()
    if days_ahead <= 0: # Target day already happened this week
        days_ahead += 7
        return d + datetime.timedelta(days_ahead)
    
def next_week(df):
    d = datetime.date.today()
    next_monday = next_weekday(d, 0) # 0 = Monday, 1=Tuesday, 2=Wednesday...
    following_monday = next_monday + datetime.timedelta(6)
    weekly_rota = df[next_monday:following_monday] #next week cut
    return weekly_rota

def load_master_rota():
    master_rota = 'wota.xlsx'

    cols = range(0,40)
    df = pd.read_excel(master_rota, dayfirst=True, skiprows=1340, usecols=cols) #load our spreadsheet
    
    current_cols = ['Date', 'Day', 'Notes', 'ConsOnCallNote1', 'ConsOnCallNote2', 'First Floor Consultant', 'Manvers Consultant', 
                'MB', 'SE', 'LYH', 'OMK', 'WO', 'MW', 'PM', 'IC', 'SB', 'CR', 
                'Date1', 'Day1', 'SprNote1', 'SprNote2', 'NoReg1', 'NoReg2', 'Chris', 'Carl', 'Justin', 'Reza', 
                'Date2', 'Day2', 'Sophie', 'Nick', 'Nicola', 'NoSHO1', 'NoSHO2', 'NoSHO3', 'Saman', 
                'Alix', 'Erin', 'Cat', 'Vytas'] #define column headings

    reg = ['Chris', 'Carl', 'Justin', 'Reza']

    assert len(df.columns) == len(current_cols) #sanity check

    df.columns = current_cols #set our column names

    df = df[['Date', 'Day', 'First Floor Consultant', 'Manvers Consultant', 
                'MB', 'SE', 'LYH', 'OMK', 'WO', 'MW', 'PM', 'IC', 'SB', 'CR', 
                'Chris', 'Carl', 'Justin', 'Reza', 
                'Sophie', 'Nick', 'Nicola', 'Saman', 
                'Alix', 'Erin', 'Cat', 'Vytas']]  #throw away columns we don't need

    df.index = df.Date #lets index on the date because its better that way
    return df

In [3]:
def FirstFloorReg(df):
    FirstFloorReg = {}
    reg = ['Chris', 'Carl', 'Justin', 'Reza']
    
    for d in df['Date']:
        for registrar in reg:
            if df[d:d][registrar][(df[d:d][registrar].astype(str) == '1')].any():
                FirstFloorReg[d] = registrar 
            elif df[d:d][registrar][(df[d:d][registrar].astype(str) == 'PT')].any():
                FirstFloorReg[d] = registrar
            elif df[d:d][registrar][(df[d:d][registrar].astype(str) == 'Take')].any():
                FirstFloorReg[d] = registrar  
                
    return FirstFloorReg

def FirstFloorJunior(df):
    FirstFloorJunior = {}
    juniors = ['Sophie','Nicola', 'Saman', 'Nick',
                'Alix', 'Erin', 'Cat', 'Vytas']
    
    for d in df['Date']:
        for junior in juniors:
            if df[d:d][junior][(df[d:d][junior].astype(str) == '1')].any():
                if d.to_pydatetime() in FirstFloorJunior:
                    FirstFloorJunior[d.to_pydatetime()].append(junior)
                else:
                    FirstFloorJunior[d.to_pydatetime()] = [junior]
            elif df[d:d][junior][(df[d:d][junior].astype(str) == 'PT')].any():
                if d.to_pydatetime() in FirstFloorJunior:
                    FirstFloorJunior[d.to_pydatetime()].append(junior)
                else:
                    FirstFloorJunior[d.to_pydatetime()] = [junior]
            elif df[d:d][junior][(df[d:d][junior].astype(str) == 'Take')].any():
                if d.to_pydatetime() in FirstFloorJunior:
                    FirstFloorJunior[d.to_pydatetime()].append(junior)
                else:
                    FirstFloorJunior[d.to_pydatetime()] = [junior]

    return FirstFloorJunior


def SecondFloorReg(df):
    SecondFloorReg = {}
    reg = ['Chris', 'Carl', 'Justin', 'Reza']
    
    for d in df['Date']:
        for registrar in reg:
            if df[d:d][registrar][(df[d:d][registrar].astype(str) == '2')].any():
                SecondFloorReg[d] = registrar 
                
    return SecondFloorReg

def SecondFloorJunior(df):
    SecondFloorJunior = {}
    juniors = ['Sophie','Nicola', 'Saman', 'Nick',
                'Alix', 'Erin', 'Cat', 'Vytas']
    
    for d in df['Date']:
        for junior in juniors:
            if df[d:d][junior][(df[d:d][junior].astype(str) == '2')].any():
                if d.to_pydatetime() in SecondFloorJunior:
                    SecondFloorJunior[d.to_pydatetime()].append(junior)
                else:
                    SecondFloorJunior[d.to_pydatetime()] = [junior]

    return SecondFloorJunior

def ClinicReg(df):
    ClinicReg = {}
    reg = ['Chris', 'Carl', 'Justin', 'Reza']
    
    for d in df['Date']:
        for registrar in reg:
            if df[d:d][registrar][(df[d:d][registrar].astype(str) == 'CLINIC')].any():
                ClinicReg[d] = registrar 
                
    return ClinicReg


def Night(df):
    Night = {}
    staff = ['MB', 'SE', 'LYH', 'OMK', 'WO', 'MW', 'PM', 'IC', 'SB', 'CR', 
                'Chris', 'Carl', 'Justin', 'Reza', 
                'Sophie', 'Nick', 'Nicola', 'Saman', 
                'Alix', 'Erin', 'Cat', 'Vytas']
    
    for d in df['Date']:
        for s in staff:
            if df[d:d][s][(df[d:d][s].astype(str) == 'Night')].any():
                if d.to_pydatetime() in Night:
                    Night[d.to_pydatetime()].append(s)
                else:
                    Night[d.to_pydatetime()] = [s]
    return Night

def ZeroDay(df):
    ZeroDay = {}
    staff = ['MB', 'SE', 'LYH', 'OMK', 'WO', 'MW', 'PM', 'IC', 'SB', 'CR', 
                'Chris', 'Carl', 'Justin', 'Reza', 
                'Sophie', 'Nick', 'Nicola', 'Saman', 
                'Alix', 'Erin', 'Cat', 'Vytas']
    
    for d in df['Date']:
        for s in staff:
            if df[d:d][s][(df[d:d][s].astype(str) == 'Zero')].any():
                if d.to_pydatetime() in ZeroDay:
                    ZeroDay[d.to_pydatetime()].append(s)
                else:
                    ZeroDay[d.to_pydatetime()] = [s]
    return ZeroDay

def AnnualLeave(df):
    AnnualLeave = {}
    staff = ['MB', 'SE', 'LYH', 'OMK', 'WO', 'MW', 'PM', 'IC', 'SB', 'CR', 
                'Chris', 'Carl', 'Justin', 'Reza', 
                'Sophie', 'Nick', 'Nicola', 'Saman', 
                'Alix', 'Erin', 'Cat', 'Vytas']
    
    for d in df['Date']:
        for s in staff:
            if df[d:d][s][(df[d:d][s].astype(str) == 'LEAVE')].any():
                if d.to_pydatetime() in AnnualLeave:
                    AnnualLeave[d.to_pydatetime()].append(s)
                else:
                    AnnualLeave[d.to_pydatetime()] = [s]

    return AnnualLeave

def StudyLeave(df):
    StudyLeave = {}
    staff = ['MB', 'SE', 'LYH', 'OMK', 'WO', 'MW', 'PM', 'IC', 'SB', 'CR', 
                'Chris', 'Carl', 'Justin', 'Reza', 
                'Sophie', 'Nick', 'Nicola', 'Saman', 
                'Alix', 'Erin', 'Cat', 'Vytas']
    
    for d in df['Date']:
        for s in staff:
            if df[d:d][s][(df[d:d][s].astype(str) == 'TRAINING DAY')].any():
                if d.to_pydatetime() in StudyLeave:
                    StudyLeave[d.to_pydatetime()].append(s)
                else:
                    StudyLeave[d.to_pydatetime()] = [s]
            elif df[d:d][s][(df[d:d][s].astype(str) == 'STUDY')].any():
                if d.to_pydatetime() in StudyLeave:
                    StudyLeave[d.to_pydatetime()].append(s)
                else:
                    StudyLeave[d.to_pydatetime()] = [s]
                    
    return StudyLeave



In [4]:
df = load_master_rota()
df = next_week(df)

In [5]:
first = (('First Floor Registrar (#1028)', FirstFloorReg(df)), 
         ('First Floor Juniors (#1656, 1016)', FirstFloorJunior(df)))

second = (('Second Floor Registrar (#1238)', SecondFloorReg(df)),
         ('Second Floor Juniors (#1208, 1138)', SecondFloorJunior(df)))

other = (('Clinic Registrar', ClinicReg(df)),
        ('Nights', Night(df)),
        ('Zero Days', ZeroDay(df)),
        ('Annual Leave', AnnualLeave(df)),
        ('Study Leave', StudyLeave(df)))

In [6]:
first = collections.OrderedDict(first)
second = collections.OrderedDict(second)
other = collections.OrderedDict(other)

a = df.ix[:,1:3].transpose()
b = pd.DataFrame(first).transpose()
c = df.ix[:,3:4].transpose()
d = pd.DataFrame(second).transpose()
e = pd.DataFrame(other).transpose()

data_frames = [a,b,c,d,e]

weeklyrota = pd.concat(data_frames).fillna('')

In [7]:
weeklyrota

,2015-11-02 00:00:00,2015-11-03 00:00:00,2015-11-04 00:00:00,2015-11-05 00:00:00,2015-11-06 00:00:00,2015-11-07 00:00:00,2015-11-08 00:00:00
Day,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
First Floor Consultant,IC,IC,IC,IC,IC,,
First Floor Registrar (#1028),Chris,Carl,Carl,,Carl,,
"First Floor Juniors (#1656, 1016)",,[Nicola],"[Nicola, Nick]","[Nicola, Nick]","[Nicola, Nick]",,
Manvers Consultant,WARDS: SB,WARDS: SB,WARDS: SB,WARDS: SB,WARDS: SB,,
Second Floor Registrar (#1238),Justin,Justin,Justin,,Justin,,
"Second Floor Juniors (#1208, 1138)",,"[Saman, Vytas]","[Saman, Alix, Vytas]","[Saman, Alix, Vytas]","[Saman, Alix, Vytas]",,
Clinic Registrar,,,,,,,
Nights,,,,,,[Alix],[Alix]
Zero Days,"[Reza, Sophie, Cat]","[Reza, Sophie, Cat]","[Reza, Sophie, Cat]","[Sophie, Cat]","[Reza, Sophie, Cat]",,


In [8]:
d = datetime.date.today()
week_starts = next_weekday(d, 0)

In [9]:
rotaname = '%s_weekly_rota.csv' % next_weekday(d, 0)

In [10]:
weeklyrota.to_csv(rotaname)